In [46]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi
from os import path
from hail.experimental import dnd

def getIfNotExists(mtPath: str, nsample = 1000, mvariants = 50_000):
    if not path.exists(mtPath):
        mt = hl.balding_nichols_model(1, nsample, mvariants)
        mt = mt.select_entries(ac = mt.GT.n_alt_alleles())
        mt.write(mtPath)
    else:
        mt = hl.methods.read_matrix_table(mtPath)
    
    return mt

# mtPath = "balding_nichols_1_1k_50k.mt"
mtPath = "balding_nichols_1_100_10k.mt"
mt = getIfNotExists(mtPath, 100, 10000)
# mt = mt.key_rows_by("locus").drop("alleles", "ancestral_af", "af")
mt.show()

,,,,,,,,,
,,0,1,2,3,4,5,6,7
locus,alleles,ac,ac,ac,ac,ac,ac,ac,ac
locus<GRCh37>,array<str>,int32,int32,int32,int32,int32,int32,int32,int32
1:1,"[""A"",""C""]",1,2,1,1,0,0,2,2
1:2,"[""A"",""C""]",1,2,1,0,2,2,2,1
1:3,"[""A"",""C""]",1,2,2,0,2,2,1,2
1:4,"[""A"",""C""]",2,1,1,1,1,2,1,2
1:5,"[""A"",""C""]",0,1,2,0,2,2,1,1
1:6,"[""A"",""C""]",0,2,1,1,1,2,0,1
1:7,"[""A"",""C""]",1,1,0,0,1,2,2,0


In [53]:
# What does it mean for there to be two column fields?
# In this case, "pop" does not provide a unique index for columns, it's always 0, since we have 1 population
mt.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
----------------------------------------
Entry fields:
    'ac': int32
----------------------------------------
Column key: ['sample_idx']
Row key: ['locus', 'alleles']
----------------------------------------


In [58]:
# This creates one row for each unique pair of (row_key, column_key) I think
# mt.entries().show(200)

In [60]:
# The column field in a hail matrix table always? contains the sample id (can we have no samples?...would make sense yes, as long as the field could act as a column index)
#mt.sample_idx.show()

In [67]:
# Current limitation ist hat dnd array doesnt support rectangles
# so number of samples in a group (columns) matches number of variants (rows)
# Not a real issue,
# block_size is the size in each dimension
darray = dnd.array(mt, "ac", block_size = 1000)
darray

2020-07-21 11:24:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-21 11:24:52 Hail: INFO: wrote table with 10 rows in 8 partitions to /tmp/yfXBkQTE4lI7Jpr86wR5lV


In [69]:
# it is blocked by some rows, and all of the columns
# so we get what we want for first pass...I think
# lets check with more samples
# Here we see that each block contains 1000 rows, and 100 columns, for 1000 variants, and all 100 samples
# (when n < block_size, n is used)
dt = darray.m.block.collect()
# We have 10 blocks, since number of samples is smaller than block_size, and there are 10k variants
print("N blocks:", len(dt))
print("Shape:", dt[0].shape)

N blocks: 10
Shape: (1000, 100)


# Regenie method

In [71]:
# From: https://glow.readthedocs.io/en/latest/tertiary/whole-genome-regression.html

# Step 1 import vcf, split multi allelics, mean impute genotypes, which means to replace missing values with the mean of non-missing values
from pyspark.sql.functions import col, lit

variants = spark.read.format('vcf').load(genotypes_vcf)
genotypes = glow.transform('split_multiallelics', variants) \
    .withColumn('values', glow.mean_substitute(glow.genotype_states(col('genotypes'))))

# Step 2 import phenotype data, and store it in a pandas dataframe called "label_df"
# also need to standardize phenotypes
# Note: this is only for continuous phenotypes in WGR
# Note: we can store these labels as entries,
# or for dnd arrays, as globals I believe
# Dimension: Y is N x 1, and in blocked form Bc X 1, where Bc is the column dim of the block
import pandas as pd

label_df = pd.read_csv(continuous_phenotypes_csv, index_col='sample_id')
label_df = ((label_df - label_df.mean())/label_df.std(ddof=0))[['Continuous_Trait_1', 'Continuous_Trait_2']]

# Step 3: import covaraites, and standardize
# Again, can store these as globals
# I believe the dimension here is N x C, where C is number of covariates
# except that if we block by samples as well as variants, this will need to be Bc X C, where Bc is the column dimension of the block
# This gets prepended to the X array, can do this using one of the numpy stacking operations
covariates = pd.read_csv(covariates_csv, index_col='sample_id')
covariates = (covariates - covariates.mean())/covariates.std(ddof=0)

# Step 4: Genotype matrix blocking
# This function creates an Br x Bc blocking of N * X
# Each block contains a certain number of rows an a certain number of columns
# They also remove rows that are completely uniform across all samples (since matrix will not be full rank if they don't)
# but I'm not sure how that affects things

# One advantage of WGR is that they allow rectangular blocks
# Description: 
# glow.wgr.functions.block_variants_and_samples creates two objects: a block genotype matrix and a sample block mapping.

# Parameters

# genotypes: Genotype DataFrame created by reading from any variant datasource supported by Glow, such as VCF. Must also include a column values containing a numeric representation of each genotype.
# sample_ids: List of sample IDs. Can be created by applying glow.wgr.functions.get_sample_ids to a genotype DataFrame.
# variants_per_block: Number of variants to include per block. We recommend 1000.
# sample_block_count: Number of sample blocks to create. We recommend 10.

# Return

# The function returns a block genotype matrix and a sample block mapping.

# Block genotype matrix

# If we imagine the block genotype matrix conceptually, we think of an NxM matrix X where each row n represents an individual sample, each column m represents a variant, and each cell (n, m) contains a genotype value for sample n at variant m. We then imagine laying a coarse grid on top of this matrix such that matrix cells within the same coarse grid cell are all assigned to the same block x. Each block x is indexed by a sample block ID (corresponding to a list of rows belonging to the block) and a header block ID (corresponding to a list of columns belonging to the block). The sample block IDs are generally just integers 0 through the number of sample blocks. The header block IDs are strings of the form ‘chr_C_block_B’, which refers to the Bth block on chromosome C. The Spark DataFrame representing this block matrix can be thought of as the transpose of each block xT all stacked one atop another. Each row represents the values from a particular column from X, for the samples corresponding to a particular sample block. The fields in the DataFrame are:

# header: A column name in the conceptual matrix X.
# size: The number of individuals in the sample block for the row.
# values: Genotype values for this header in this sample block. If the matrix is sparse, contains only non-zero values.
# header_block: An ID assigned to the block x containing this header.
# sample_block: An ID assigned to the block x containing the group of samples represented on this row.
# position: An integer assigned to this header that specifies the correct sort order for the headers in this block.
# mu: The mean of the genotype calls for this header.
# sig: The standard deviation of the genotype calls for this header.
# Sample block mapping

# The sample block mapping consists of key-value pairs, where each key is a sample block ID and each value is a list of sample IDs contained in that sample block.

# The order of these IDs match the order of the values arrays in the block genotype DataFrame.
from glow.wgr.linear_model import RidgeReducer, RidgeRegression
from glow.wgr.functions import block_variants_and_samples, get_sample_ids
from pyspark.sql.functions import col, lit

variants_per_block = 1000
sample_block_count = 10
sample_ids = get_sample_ids(genotypes)
block_df, sample_blocks = block_variants_and_samples(
    genotypes, sample_ids, variants_per_block, sample_block_count)


# Step 5: dimensionality reduction
# This runs RidgeReducer
# This initializes some alphas
reducer = RidgeReducer()


## Remaining questions:
# 1) how do we do the leave one out ? Will we need a mapping between blocks and chromosomes?


NameError: name 'spark' is not defined

In [108]:
# ok, if too small then it blocks off the samples
# Ok, we have 200 rows, of block size 50, 50 * 200 == 10_000, number of variants
# columns we have 2 of, 2 * 50 == 100

# interestingly when I transpose, and then take M, I get wrong number of columns
max(darray2.T.m.c.collect())

1

In [110]:
darray2.m = darray2.m.annotate(x = hl.nd.inv(darray2.m.block.T @ darray.m.block))

In [111]:
darray2.m.show()

+-------+-------+
|     r |     c |
+-------+-------+
| int32 | int32 |
+-------+-------+
|     0 |     0 |
|     0 |     1 |
|     1 |     0 |
|     1 |     1 |
|     2 |     0 |
|     2 |     1 |
|     3 |     0 |
|     3 |     1 |
|     4 |     0 |
|     4 |     1 |
+-------+-------+

+-------------------------------------------------------------------------------------------------------------------------------------+
| block                                                                                                                               |
+-------------------------------------------------------------------------------------------------------------------------------------+
| ndarray<int32, 2>                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------+
| ndarray{shape=(50, 50), data=[[1, 2, 1, 1, 0, 0, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1... |
| ndarray{shape=(50, 50), data=[[2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1... |
| ndarray{shape=(50, 50), data=[[0, 0, 2, 0, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 1, 2, 0, 0, 0, 2, 1... |
| ndarray{shape=(50, 50), data=[[0, 2, 1, 1, 2, 0, 2, 2, 1, 2, 0, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1... |
| ndarray{shape=(50, 50), data=[[2, 0, 0, 2, 2, 1, 2, 1, 1, 2, 1, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1... |
| ndarray{shape=(50, 50), data=[[1, 1, 2, 1, 1, 2, 1, 2, 0, 0, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 0, 2, 1, 0, 1, 1, 2... |
| ndarray{shape=(50, 50), data=[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0... |
| ndarray{shape=(50, 50), data=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0... |
| ndarray{shape=(50, 50), data=[[1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 2, 1, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 2... |
| ndarray{shape=(50, 50), data=[[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1, 0... |
+-------------------------------------------------------------------------------------------------------------------------------------+

+-------------------------------------------------------------------------------------------------------------------------------------+
| x                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------+
| ndarray<float64, 2>                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------+
| ndarray{shape=(50, 50), data=[[6.11666728640559E12, -4.0720567647158E12, -1.026922616767063E13, -2.720389277036237E12, 1.2702236... |
| ndarray{shape=(50, 50), data=[[-1.8593447652394547E10, -1.6371754279467676E11, -5.533052235121338E10, 3.4318712767463965E11, -1.... |
| ndarray{shape=(50, 50), data=[[3.7398886480018567, 2.3064579359295916, -1.0448263755576823, 0.42915327381921564, 1.8070936952477... |
| ndarray{shape=(50, 50), data=[[4.9437949150167695, 1.699091432891878, -3.7821480286354103, -3.715575056314618, 2.943890055724182... |
| ndarray{shape=(50, 50), data=[[18.980072813084476, -3.0726292214910513, 6.226889391537279, -33.77435610694214, 9.284162242414398... |
| ndarray{shape=(50, 50), data=[[275.24486057434814, -178.80961968557446, 89.13508739782884, 75.2656783893061, 109.97486572787525,... |
| ndarray{shape=(50, 50), data=[[2.245

In [2]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
----------------------------------------
Entry fields:
    'n_alt': float64
----------------------------------------
Column key: ['sample_idx']
Row key: ['locus', 'alleles']
----------------------------------------


In [3]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.show()

,,,,,,,,
,,0,1,2,3,4,5,6
locus,alleles,n_alt,n_alt,n_alt,n_alt,n_alt,n_alt,n_alt
locus<GRCh37>,array<str>,float64,float64,float64,float64,float64,float64,float64
1:1,"[""A"",""C""]",1.00e+00,2.00e+00,1.00e+00,1.00e+00,0.00e+00,0.00e+00,2.00e+00
1:2,"[""A"",""C""]",2.00e+00,2.00e+00,1.00e+00,0.00e+00,2.00e+00,2.00e+00,2.00e+00
1:3,"[""A"",""C""]",0.00e+00,1.00e+00,2.00e+00,0.00e+00,1.00e+00,2.00e+00,1.00e+00
1:4,"[""A"",""C""]",2.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00
1:5,"[""A"",""C""]",0.00e+00,0.00e+00,1.00e+00,0.00e+00,1.00e+00,1.00e+00,1.00e+00
1:6,"[""A"",""C""]",0.00e+00,2.00e+00,2.00e+00,1.00e+00,2.00e+00,2.00e+00,0.00e+00
1:7,"[""A"",""C""]",0.00e+00,1.00e+00,0.00e+00,0.00e+00,1.00e+00,1.00e+00,1.00e+00


In [4]:
mt.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
----------------------------------------
Entry fields:
    'n_alt': float64
----------------------------------------
Column key: ['sample_idx']
Row key: ['locus', 'alleles']
----------------------------------------


In [5]:
mt.n_partitions()

8

In [7]:
wd = "/Users/alexkotlar/projects/hail/hail/python/test/data"
table = (hl.import_table(f"{wd}/1kg_annotations.txt", impute=True).key_by('Sample'))
table

2020-07-20 20:12:07 Hail: INFO: Reading table to impute column types
2020-07-20 20:12:08 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)


In [8]:
table.show()

,,,,,
Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",false,false,4
"""HG00097""","""GBR""","""EUR""",true,true,4
"""HG00098""","""GBR""","""EUR""",false,false,5
"""HG00099""","""GBR""","""EUR""",true,false,4
"""HG00100""","""GBR""","""EUR""",true,false,5
"""HG00101""","""GBR""","""EUR""",false,true,1
"""HG00102""","""GBR""","""EUR""",true,true,6
"""HG00103""","""GBR""","""EUR""",false,true,5


In [9]:
print(mt.col.dtype)

struct{sample_idx: int32, pop: int32}


In [10]:
hl.utils.get_1kg('data/')


2020-07-20 20:13:04 Hail: INFO: downloading 1KG VCF ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz
2020-07-20 20:13:05 Hail: INFO: importing VCF and writing to matrix table...
2020-07-20 20:13:05 Hail: INFO: Coerced sorted dataset
2020-07-20 20:13:07 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 16 partitions to data/1kg.mt
2020-07-20 20:13:07 Hail: INFO: downloading 1KG annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt
2020-07-20 20:13:07 Hail: INFO: downloading Ensembl gene annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/ensembl_gene_annotations.txt
2020-07-20 20:13:08 Hail: INFO: Done!


In [11]:
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)


2020-07-20 20:13:13 Hail: INFO: Coerced sorted dataset
2020-07-20 20:13:17 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 1 partition to data/1kg.mt


In [12]:
mt = hl.read_matrix_table('data/1kg.mt')


In [13]:
mt.rows().select().show(5)


,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


In [14]:
mt.count_rows()

10879

In [22]:
rwd = "/Users/alexkotlar/projects/regenie/example"
covars = hl.import_table(f"{rwd}/covariates.txt", impute=True, delimiter=" ")
phenos = hl.import_table(f"{rwd}/phenotype.txt", impute=True, delimiter=" ")
covariates

2020-07-20 20:23:00 Hail: INFO: Reading table to impute column types
2020-07-20 20:23:00 Hail: INFO: Finished type imputation
  Loading field 'FID' as type int32 (imputed)
  Loading field 'IID' as type int32 (imputed)
  Loading field 'V1' as type float64 (imputed)
  Loading field 'V2' as type float64 (imputed)
  Loading field 'V3' as type float64 (imputed)


In [23]:
covariates.show()

,,,,
FID,IID,V1,V2,V3
int32,int32,float64,float64,float64
1,1,1.47e+00,1.94e+00,1.53e-01
2,2,-1.22e+00,-1.63e+00,-1.90e-01
3,3,7.12e-02,8.64e-02,1.43e-01
4,4,-1.92e+00,-1.08e+00,2.09e-01
5,5,-3.73e-02,-1.03e+00,1.14e-01
6,6,1.13e+00,1.16e+00,-1.02e+00
7,7,1.25e-01,-5.61e-01,2.66e-01
8,8,2.31e+00,-2.08e+00,3.12e-01


In [24]:
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    }
----------------------------------------
Entry fields:
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'PL': array<int32>
    'n_alt': float64
---------------------------------------

In [25]:
mt.show()

+---------------+------------+--------------+------------+------------+--------------+---------------+
| locus         | alleles    | HG00096.AD   | HG00096.DP | HG00096.GQ | HG00096.PL   | HG00096.n_alt |
+---------------+------------+--------------+------------+------------+--------------+---------------+
| locus<GRCh37> | array<str> | array<int32> |      int32 |      int32 | array<int32> |       float64 |
+---------------+------------+--------------+------------+------------+--------------+---------------+
| 1:904165      | ["G","A"]  | [4,0]        |          4 |         12 | [0,12,147]   |      0.00e+00 |
| 1:909917      | ["G","A"]  | [4,0]        |          4 |         12 | [0,12,160]   |      0.00e+00 |
| 1:986963      | ["C","T"]  | [3,0]        |          3 |          9 | [0,9,98]     |      0.00e+00 |
| 1:1563691     | ["T","G"]  | NA           |         NA |         NA | NA           |            NA |
| 1:1707740     | ["T","G"]  | [2,3]        |          5 |         67 | [82,0,67]    |      1.00e+00 |
| 1:2252970     | ["C","T"]  | [4,0]        |          4 |         12 | [0,12,162]   |      0.00e+00 |
| 1:2284195     | ["T","C"]  | [0,1]        |          1 |          3 | [38,3,0]     |      2.00e+00 |
| 1:2779043     | ["T","C"]  | [2,2]        |          4 |         33 | [33,0,74]    |      1.00e+00 |
| 1:2944527     | ["G","A"]  | [2,0]        |          2 |          6 | [0,6,76]     |      0.00e+00 |
| 1:3761547     | ["C","A"]  | [4,0]        |          4 |         12 | [0,12,176]   |      0.00e+00 |
+---------------+------------+--------------+------------+------------+--------------+---------------+
showing top 10 rows
showing the first 1 of 284 columns

In [26]:

# Turn MatrixTable into Table

ht = mt.localize_entries("ent", "sample")
ht.describe()

----------------------------------------
Global fields:
    'sample': array<struct {
        s: str
    }> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    } 
    'ent': array<struct {
        AD: array<int32>, 
        DP: int32, 
        GQ: int32, 
        PL: array<int32>, 
        n_alt: float64
    }> 
----------------------------------------
Key: ['locus', 'allel

2020-07-20 22:33:54 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']


In [27]:
ht.show()

+---------------+------------+------+----------+----------+--------------+----------------+---------+-------------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      | info.AF        | info.AN | info.BaseQRankSum |
+---------------+------------+------+----------+----------+--------------+----------------+---------+-------------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> | array<float64> |   int32 |           float64 |
+---------------+------------+------+----------+----------+--------------+----------------+---------+-------------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        | [1.03e-01]     |    5020 |         -3.39e+00 |
| 1:909917      | ["G","A"]  | NA   | 1.58e+03 | NA       | [18]         | [3.73e-03]     |    4830 |         -1.48e+00 |
| 1:986963      | ["C","T"]  | NA   | 3.98e+02 | NA       | [5]          | [1.09e-03]     |    4588 |          1.25e+00 |
| 1:1563691     | ["T","G"]  | NA   | 1.09e+03 | NA       | [64]         | [1.30e-02]     |    4766 |         -3.87e+01 |
| 1:1707740     | ["T","G"]  | NA   | 9.35e+04 | NA       | [997]        | [1.98e-01]     |    5034 |         -4.04e+01 |
| 1:2252970     | ["C","T"]  | NA   | 7.36e+02 | NA       | [6]          | [1.28e-03]     |    4682 |         -1.22e+00 |
| 1:2284195     | ["T","C"]  | NA   | 1.42e+05 | NA       | [1559]       | [3.12e-01]     |    4990 |         -4.60e+01 |
| 1:2779043     | ["T","C"]  | NA   | 2.89e+05 | NA       | [3532]       | [7.26e-01]     |    4866 |          1.74e+01 |
| 1:2944527     | ["G","A"]  | NA   | 1.24e+05 | NA       | [1206]       | [2.45e-01]     |    4928 |          6.30e-02 |
| 1:3761547     | ["C","A"]  | NA   | 1.61e+03 | NA       | [30]         | [5.95e-03]     |    5044 |         -4.47e+00 |
+---------------+------------+------+----------+----------+--------------+----------------+---------+-------------------+

+----------------------+---------+---------+----------+---------------------+----------------------+--------------+----------------+
| info.ClippingRankSum | info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff | info.MLEAC   | info.MLEAF     |
+----------------------+---------+---------+----------+---------------------+----------------------+--------------+----------------+
|              float64 |   int32 |    bool |  float64 |             float64 |              float64 | array<int32> | array<float64> |
+----------------------+---------+---------+----------+---------------------+----------------------+--------------+----------------+
|            -1.70e-01 |   17827 |   false | 2.23e+00 |                  NA |             9.88e-02 | [514]        | [1.02e-01]     |
|             1.26e-01 |   14671 |   false | 5.52e+00 |                  NA |            -5.00e-04 | [15]         | [3.11e-03]     |
|            -3.77e+00 |   12398 |   false | 8.34e-01 |                  NA |             1.26e-02 | [3]          | [6.54e-04]     |
|            -5.39e+00 |   15357 |   false | 1.90e+03 |                  NA |             2.70e-02 | [22]         | [4.62e-03]     |
|            -2.87e-01 |   19902 |   false | 3.31e+00 |                  NA |             3.87e-02 | [983]        | [1.95e-01]     |
|             1.79e+00 |   14900 |   false | 2.82e+00 |                  NA |            -8.20e-03 | [6]          | [1.28e-03]     |
|             3.50e-01 |   18176 |   false | 2.95e+00 |                  NA |             9.25e-02 | [1552]       | [3.11e-01]     |
|             2.13e+00 |   12878 |   false | 2.55e+01 |                  NA |             1.44e-01 | [3545]       | [7.29e-01]     |
|            -6.55e-01 |   17698 |   false | 4.49e-01 |                  NA |             1.23e-01 | [1192]       | [2.42e-01]     |
|            -8.82e+00 |   16845 |   false | 2.06e+00 |                  NA |            -4.70e-03 | [28]         | [5.55e-03]     |
+----------------------+---------+---------+--------

In [28]:
# Functions for operating with Tables of ndarrays in Hail (from Tim)

from hail.expr import Expression, ExpressionException, \
    expr_float64, expr_call, expr_any, expr_numeric, expr_array, \
    expr_locus, \
    analyze, check_entry_indexed, check_row_indexed, \
    matrix_table_source, table_source

# Only groups by rows, NOT COLUMNS
def matrix_table_to_table_of_ndarrays(field, group_size, tmp_path = '/tmp/nd_table.ht'):
    """

    The returned table has two fields: 'row_group_number' and 'ndarray'.

    Examples
    --------
    >>> ht = matrix_table_to_table_of_ndarrays(mt.GT.n_alt_alleles(), 100)

    Parameters
    ----------
    field
    group_size
    tmp_path

    Returns
    -------

    """
    mt = matrix_table_source('matrix_table_to_table_of_ndarrays/x', field)
    mt = mt.select_entries(x = field)
    ht = mt.localize_entries(entries_array_field_name='entries')
    # now ht.entries is an array of structs with one field, x

    # we'll also want to mean-impute/variance-normalize/etc here
    ht = ht.select(xs = ht.entries.map(lambda e: e['x']))
    # now ht.xs is an array of float64

    # now need to produce groups of G
    ht = ht.add_index()
    ht = ht.group_by(row_group_number=ht.idx // group_size) \
        .aggregate(ndarray=hl.nd.array(hl.agg.collect(ht.xs)))
    # may require a .T on ndarray

    return ht.checkpoint(tmp_path, overwrite=True)

def chunk_ndarray(a, group_size):
    """Chunks a NDarray along the first axis in chunks of `group_size`.
    Parameters
    ----------
    a
    group_size

    Returns
    -------

    """
    n_groups = a.shape[0] // group_size
    groups = []
    for i in range(a.shape[0] // group_size):
        start = i * group_size
        end = (i + 1) * group_size
        groups.append(a[start:end, :])
    return groups

2020-07-20 23:05:40 Hail: INFO: balding_nichols_model: generating genotypes for 1 populations, 1000 samples, and 50000 variants...
2020-07-20 23:05:41 Hail: INFO: Coerced sorted dataset
2020-07-20 23:05:44 Hail: INFO: wrote matrix table with 50000 rows and 1000 columns in 8 partitions to balding_nichols_1_1k_50k.mt


In [ ]:
hl.balding_nichols_model?

In [44]:
bnm_mt.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['sample_idx']
Row key: ['locus', 'alleles']
----------------------------------------


TypeError: array() missing 2 required positional arguments: 'mt' and 'entry_field'

In [62]:
mt.rows().show()

2020-07-20 23:06:15 Hail: INFO: Coerced sorted dataset


,,,
locus,alleles,ancestral_af,af
locus<GRCh37>,array<str>,float64,array<float64>
1:1,"[""A"",""C""]",5.39e-01,[6.04e-01]
1:2,"[""A"",""C""]",6.72e-01,[4.78e-01]
1:3,"[""A"",""C""]",5.82e-01,[2.97e-01]
1:4,"[""A"",""C""]",5.36e-01,[6.83e-01]
1:5,"[""A"",""C""]",4.39e-01,[6.18e-01]
1:6,"[""A"",""C""]",6.17e-01,[7.42e-01]
1:7,"[""A"",""C""]",4.50e-01,[7.37e-01]
1:8,"[""A"",""C""]",8.13e-01,[9.04e-01]


In [49]:
test = dnd.array(bnm_mt, "GT")

2020-07-20 22:48:35 Hail: INFO: Coerced sorted dataset
2020-07-20 22:48:36 Hail: INFO: Coerced sorted dataset
2020-07-20 22:48:36 Hail: INFO: wrote table with 1 row in 1 partition to /tmp/58NIpTqyxXnpIuoFwwipsz


In [50]:
test.show()

+-------+-------+
|     r |     c |
+-------+-------+
| int32 | int32 |
+-------+-------+
|     0 |     0 |
+-------+-------+

+-------------------------------------------------------------------------------------------------------------------------------------+
| block                                                                                                                               |
+-------------------------------------------------------------------------------------------------------------------------------------+
| ndarray<call, 2>                                                                                                                    |
+-------------------------------------------------------------------------------------------------------------------------------------+
| ndarray{shape=(1000, 100), data=[[4, 4, 12, 20, 12, 4, 12, 12, 12, 12, 12, 12, 20, 4, 4, 12, 12, 12, 12, 4, 20, 4, 12, 20, 20, 4... |
+-------------------------------------------------------------------------------------------------------------------------------------+

In [65]:
mt.show()

2020-07-20 23:08:24 Hail: INFO: Coerced sorted dataset


,,,,,,,,,,
,,0,1,2,3,4,5,6,7,8
locus,alleles,GT,GT,GT,GT,GT,GT,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call,call,call,call,call,call,call
1:1,"[""A"",""C""]",1/1,1/1,0/1,1/1,0/0,0/1,0/1,0/1,1/1
1:2,"[""A"",""C""]",0/1,0/0,1/1,1/1,1/1,1/1,0/1,0/0,0/0
1:3,"[""A"",""C""]",1/1,1/1,0/0,0/0,0/0,0/1,0/0,0/1,0/0
1:4,"[""A"",""C""]",0/1,0/1,1/1,0/1,1/1,1/1,0/1,1/1,1/1
1:5,"[""A"",""C""]",0/0,0/1,0/1,0/1,1/1,0/1,0/1,0/1,1/1
1:6,"[""A"",""C""]",1/1,0/1,1/1,1/1,0/0,0/1,1/1,1/1,0/1
1:7,"[""A"",""C""]",1/1,1/1,0/1,0/1,1/1,0/1,0/1,0/0,1/1


In [55]:
bnm_mt.GT.

2020-07-20 23:03:57 Hail: INFO: Coerced sorted dataset


,,,
locus,alleles,ancestral_af,af
locus<GRCh37>,array<str>,float64,array<float64>
1:1,"[""A"",""C""]",5.39e-01,"[3.79e-01,5.76e-01,4.01e-01]"
1:2,"[""A"",""C""]",6.72e-01,"[6.88e-01,5.26e-01,6.45e-01]"
1:3,"[""A"",""C""]",5.82e-01,"[5.81e-01,6.67e-01,4.28e-01]"
1:4,"[""A"",""C""]",5.36e-01,"[3.35e-01,4.08e-01,4.99e-01]"
1:5,"[""A"",""C""]",4.39e-01,"[6.38e-01,5.70e-01,7.86e-02]"
1:6,"[""A"",""C""]",6.17e-01,"[6.32e-01,5.64e-01,6.65e-01]"
1:7,"[""A"",""C""]",4.50e-01,"[4.03e-01,1.34e-01,4.40e-01]"
1:8,"[""A"",""C""]",8.13e-01,"[5.90e-01,8.23e-01,8.77e-01]"


In [ ]:
mt.show()

In [68]:
mt.GT.show()

2020-07-20 23:10:53 Hail: INFO: Coerced sorted dataset


,,,,,,,,,,
,,0,1,2,3,4,5,6,7,8
locus,alleles,GT,GT,GT,GT,GT,GT,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call,call,call,call,call,call,call
1:1,"[""A"",""C""]",1/1,1/1,0/1,1/1,0/0,0/1,0/1,0/1,1/1
1:2,"[""A"",""C""]",0/1,0/0,1/1,1/1,1/1,1/1,0/1,0/0,0/0
1:3,"[""A"",""C""]",1/1,1/1,0/0,0/0,0/0,0/1,0/0,0/1,0/0
1:4,"[""A"",""C""]",0/1,0/1,1/1,0/1,1/1,1/1,0/1,1/1,1/1
1:5,"[""A"",""C""]",0/0,0/1,0/1,0/1,1/1,0/1,0/1,0/1,1/1
1:6,"[""A"",""C""]",1/1,0/1,1/1,1/1,0/0,0/1,1/1,1/1,0/1
1:7,"[""A"",""C""]",1/1,1/1,0/1,0/1,1/1,0/1,0/1,0/0,1/1


In [21]:
df = hl.utils.range_table(100)
df.show()

""
idx
int32
0
1
2
3
4
5
6
7


In [23]:
n_partitions = mt.n_partitions()
block_size = 4096

In [25]:
import json
fast_codec_spec = json.dumps({
        "name": "BlockingBufferSpec",
        "blockSize": 64 * 1024,
        "child": {
            "name": "LZ4FastBlockBufferSpec",
            "blockSize": 64 * 1024,
            "child": {
                "name": "StreamBlockBufferSpec"}}})

In [26]:
from hail.utils.java import Env

In [27]:
n_rows, n_cols = mt.count()
n_block_rows = (n_rows + block_size - 1) // block_size
n_block_cols = (n_cols + block_size - 1) // block_size
entries, cols, row_index, col_blocks = (Env.get_uid() for _ in range(4))

In [28]:
entries

'__uid_44'

In [29]:
cols

'__uid_45'

In [30]:
row_index

'__uid_46'

In [31]:
n_block_rows

3

In [32]:
n_block_cols

1

In [43]:
np.array([[0,1,2], [3,4,5]])

array([[0, 1, 2],
       [3, 4, 5]])

In [44]:
nd = np.array([[0,1,2], [2,3,4]])

n_rows = nd.shape[0]

In [45]:
n_rows

2

In [46]:
nd[3 // n_rows, 3 % n_rows]

3

In [47]:
4 // n_rows

2

In [48]:
4 % n_rows

0

In [52]:

nd = np.array([[0,1,2], [3,4,5]])
nd

array([[0, 1, 2],
       [3, 4, 5]])

In [53]:
nd[1,1]

4

In [54]:
4 % 2

0

In [55]:
4 // 2

2

In [56]:
nd[4 % 2, 4 // 2]

2

In [80]:
i = 1
n_rows = 2
n_cols = 3

nd.T[i % n_rows, i // n_cols]

1

In [76]:
nd[i // n_cols, i % n_cols]

1

In [60]:
4 // 3

1

In [67]:
nd.shape

(2, 3)

In [70]:
10 % 3

1

In [75]:
10 - 3 * (10 // 3)

1

In [81]:
def eye(N, M=None, dtype=hl.tfloat64):
    """
    Construct a 2-D :class:`.NDArrayExpression` with ones on the *main* diagonal
    and zeros elsewhere.

    Parameters
    ----------
    N : :class:`.NumericExpression` or Python number
      Number of rows in the output.
    M : :class:`.NumericExpression` or Python number, optional
      Number of columns in the output. If None, defaults to `N`.
    dtype : numeric :class:`.HailType`, optional
      Element type of the returned array. Defaults to :py:data:`.tfloat64`

    Returns
    -------
    I : :class:`.NDArrayExpression` representing a Hail ndarray of shape (N,M)
      An ndarray whose elements are equal to one on the main diagonal, zeroes elsewhere.

    See Also
    --------
    :func:`.identity`
    :func:`.diagonal`

    Examples
    --------
    >>> hl.eval(hl.nd.eye(3))
    array([[1., 0., 0.],
           [0., 1., 0.],
           [0., 0., 1.]])
    >>> hl.eval(hl.nd.eye(2, 5, dtype=hl.tint32))
    array([[1, 0, 0, 0, 0],
           [0, 1, 0, 0, 0]], dtype=int32)
    """

    n_row = hl.int32(N)
    if M is None:
        n_col = n_row
    else:
        n_col = hl.int32(M)

    return hl.nd.array(hl.range(0, n_row * n_col).map(
        lambda i: hl.cond((i // n_col) == (i % n_col),
                          hl.literal(1, dtype),
                          hl.literal(0, dtype))
    )).reshape((n_row, n_col))

In [86]:
for i in range(13):
    for y in range(13):
        assert(np.allclose(hl.eval(eye(i,y)), np.eye(i,y)))

In [91]:
i = 5
i - (i // n_cols) * n_cols

2

In [3]:
import numpy as np
import hail as hl
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
ah = hl.nd.array([1,2,3])
bh = hl.nd.array([2,3,4])

ah2 = hl.nd.array([[1], [2], [3]])
bh2 = hl.nd.array([[2], [3], [4]])
print(ah.ndim)
print("stfuff")
# print(hl.eval(hl.nah._broadcast(2)))
print(hl.eval(hl.nd.vstack((ah,bh))))
print(hl.eval(hl.nd.vstack((ah2,bh2))))
print("past")
print(hl.eval(hl.nd.concatenate((ah,bh),axis=0)))
print(np.vstack((a,b)))
print(hl.eval(ah.shape == bh.shape))
# print(hl.eval(hl.nd.resha))
print(ah.shape)
print(hl.array((ah,bh)))

1
stfuff
[[1 2 3]
 [2 3 4]]
[[1]
 [2]
 [3]
 [2]
 [3]
 [4]]
past
[1 2 3 2 3 4]
[[1 2 3]
 [2 3 4]]
True
<TupleExpression of type tuple(int64)>


TypeError: array: parameter 'collection': expected expression of type set<any> or array<any> or dict<('any', 'any')>, found tuple: (<NDArrayNumericExpression of type ndarray<int32, 1>>, <NDArrayNumericExpression of type ndarray<int32, 1>>)

In [5]:
import hail as hl
import numpy as np
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])

seq = (a,b)
assert(np.allclose(hl.eval(hl.nd.vstack(seq)), np.vstack(seq)))

a = np.array([[1], [2], [3]])
b = np.array([[2], [3], [4]])
seq = (a,b)
assert(np.allclose(hl.eval(hl.nd.vstack(seq)), np.vstack(seq)))

In [181]:

def _broadcast(nd, n_output_dims, axis=0):
        assert nd.ndim < n_output_dims
        print("nd.ndim", nd.ndim)
        # Right-align existing dimensions and start prepending new ones
        # to the left: e.g. [0, 1] -> [3, 2, 0, 1]
        # Based off numpy broadcasting with the assumption that everything
        # can be thought to have an infinite number of 1-length dimensions
        # prepended
        old_dims = range(nd.ndim)
        if axis == 1:
            new_dims = range(nd.ndim, n_output_dims)
        else:
            new_dim = range(nd.ndim, n_output_dims)
        print("old dims", old_dims)
        print("new_dims", new_dims)
        idx_mapping = list(reversed(new_dims)) + list(old_dims)
        print(idx_mapping)

In [182]:
_broadcast(x, 2)

old dims range(0, 1)
new_dims range(1, 2)
[1, 0]


In [169]:
ah.ndim

1

In [178]:
x = np.array([1, 2])
y = np.expand_dims(x, axis=1)
y.shape

(2, 1)

In [6]:
x = hl.nd.array([1., 2.])
y = hl.nd.array([3., 4.])
hl.eval(hl.nd.concatenate((x, y), axis=0))

array([1., 2., 3., 4.])

In [10]:
np.vstack(([1,2,3],[4,5,6]))

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
a = np.array([[1],[2],[3]])
b = np.array([[2],[3],[4]])

np.hstack((a,b))

array([[1, 2],
       [2, 3],
       [3, 4]])

In [13]:
hl.eval(hl.nd.vstack((a,b)))

array([[1],
       [2],
       [3],
       [2],
       [3],
       [4]])

In [19]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])

seq = (a,b)
assert(np.array_equal(hl.eval(hl.nd.vstack(seq)), np.vstack(seq)))

In [25]:
for i in range(13):
            for y in range(13):
                assert(np.array_equal(hl.eval(hl.nd.eye(i,y)), np.eye(i,y)))

In [21]:
x = np.array([[1., 2.], [3., 4.]])
y = np.array([[5.], [6.]])
np_res = np.concatenate([x, y], axis=1)

res = hl.eval(hl.nd.concatenate([x, y], axis=1))
assert np.array_equal(np_res, res)

In [28]:
c = np.random.randn(5, 5)
d = np.linalg.inv(c)
dhail = hl.eval(hl.nd.inv(c))
assert np.allclose(dhail, d)

In [74]:
hl.eval(hl.nd.eye(2, 5, dtype=hl.tint32))

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]], dtype=int32)

In [75]:
x = hl.nd.array([[1., 2.], [3., 4.]])
y = hl.nd.array([[5.], [6.]])
res = hl.nd.concatenate([x, y], axis=1)
hl.eval(res)

array([[1., 2., 5.],
       [3., 4., 6.]])